In [0]:
df_sales = spark.read.csv("/FileStore/tables/sales_data.csv", header=True, inferSchema=True)
df_store = spark.read.csv("/FileStore/tables/store_data.csv", header=True, inferSchema=True)

display(df_store)

In [0]:
display(df_sales)

In [0]:
from pyspark.sql.functions import *

df1 = df_sales.filter(df_sales['total_amount'] > 0)  # eliminar valores negativos e incorretos (erros)
df1 = df1.dropDuplicates()
df2 = df1.fillna({'sale_date': '2025-05-22'}) # setar data de vendas nulas para data atual. feito pois nao era necessario filtro/dt
df3 = df2.filter(col("sale_id").isNotNull()) # remaneja os valores nulos
df4 = df3.withColumn("year_sale", year(col("sale_date"))) # cria coluna "year" com o ano de cada venda

display(df4)



In [0]:
df_store_final = df_store.filter(col("store_id").isNotNull())
avg_size = df_store_final.select(avg("store_size")).first()[0] # calcula avg da coluna size
df_store_final = df_store_final.fillna({'store_size': avg_size}) # seta na coluna size os valores nulos com o avg
df_store_final = df_store_final.fillna({'open_date': '1910-05-22'}) # seta na coluna open_data valores antigos

display(df_store_final)


In [0]:
df_combined = df4.join(df_store_final, on='store_id', how='inner') # joins dos 2 dfs
display(df_combined)

In [0]:
df_total_store = df_combined.groupBy("store_id").agg(round(sum("total_amount")).alias("total_amount"))
display(df_total_store) # total vendas/loja

store_id,total_amount
8.0,67785.0
7.0,55415.0
18.0,43247.0
1.0,43767.0
4.0,47623.0
11.0,69091.0
14.0,44312.0
3.0,49463.0
19.0,58533.0
2.0,66691.0


In [0]:
df_sales_year = df_combined.groupBy("year_sale").agg(round(sum("total_amount")).alias("total_amount"))
display(df_sales_year) # total vendas/ano

year_sale,total_amount
2025,104295.0
2023,692.0
2022,292422.0
2020,304396.0
2021,301965.0


In [0]:
df_product_sales = df_combined.groupBy("product_id").agg(round(sum("total_amount")).alias("total_amount"))

display(df_product_sales) # total vendas/produto

product_id,total_amount
31,7682.0
34,15438.0
28,27685.0
27,13279.0
26,19600.0
44,17937.0
12,26458.0
22,21142.0
47,16451.0
1,18854.0


In [0]:
top5_stores = df_total_store.orderBy(col("total_amount").desc()).limit(5)
top5_products = df_product_sales.orderBy(col("total_amount").desc()).limit(5)

display(top5_stores)
display(top5_products)


store_id,total_amount
13.0,76172.0
10.0,75254.0
11.0,69091.0
8.0,67785.0
2.0,66691.0


product_id,total_amount
15,39824.0
46,37874.0
11,33555.0
14,33054.0
32,32385.0
